<a href="https://colab.research.google.com/github/ShreyasKale01/Dynamic-Parking-Price/blob/main/Dynamic_Pricing_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

STEP 1

In [ ]:
!mkdir -p project/data


In [ ]:
!mkdir -p project/modules

In [ ]:
!mkdir -p project/report

In [ ]:
%%writefile project/modules/model1.py
def model1(prev_price, occupancy, capacity, alpha=2):
    occ_rate = occupancy / capacity
    new_price = prev_price + alpha * occ_rate
    return max(5, min(new_price, 40))


Overwriting project/modules/model1.py


In [ ]:
%%writefile project/modules/model2.py
def model2(norm_demand, base_price=10, lam=0.5):
    price = base_price * (1 + lam * norm_demand)
    return max(5, min(price, 20))


Overwriting project/modules/model2.py


In [ ]:
%%writefile project/modules/model3.py
import numpy as np

def model3(my_price, competitor_prices):
    if not competitor_prices:
        return my_price
    avg_comp = np.mean(competitor_prices)
    if my_price > avg_comp:
        return my_price - 2
    elif my_price < avg_comp:
        return my_price + 2
    else:
        return my_price


Overwriting project/modules/model3.py


In [ ]:
%%writefile project/modules/utils.py
import numpy as np

def haversine(lat1, lon1, lat2, lon2):
    R = 6371
    dlat = np.radians(lat2 - lat1)
    dlon = np.radians(lon2 - lon1)
    a = np.sin(dlat/2)**2 + np.cos(np.radians(lat1))*np.cos(np.radians(lat2))*np.sin(dlon/2)**2
    return 2 * R * np.arcsin(np.sqrt(a))


Overwriting project/modules/utils.py


In [ ]:
%%writefile project/report/draft.md
# Dynamic Pricing for Urban Parking Lots – Report Draft


Overwriting project/report/draft.md


#STEP 2

In [ ]:
import sys
sys.path.append('/content/project/modules')

from model1 import model1
from model2 import model2
from model3 import model3
from utils import haversine


In [ ]:
import pandas as pd

df = pd.read_csv('/content/project/data/dataset.csv')
df.head()

,ID,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime
0,0,BHMBCCMKT01,577,26.144536,91.736172,61,car,low,1,0,04-10-2016,07:59:00
1,1,BHMBCCMKT01,577,26.144536,91.736172,64,car,low,1,0,04-10-2016,08:25:00
2,2,BHMBCCMKT01,577,26.144536,91.736172,80,car,low,2,0,04-10-2016,08:59:00
3,3,BHMBCCMKT01,577,26.144536,91.736172,107,car,low,2,0,04-10-2016,09:32:00
4,4,BHMBCCMKT01,577,26.144536,91.736172,150,bike,low,2,0,04-10-2016,09:59:00


In [ ]:
df.columns

Index(['ID', 'SystemCodeNumber', 'Capacity', 'Latitude', 'Longitude',
       'Occupancy', 'VehicleType', 'TrafficConditionNearby', 'QueueLength',
       'IsSpecialDay', 'LastUpdatedDate', 'LastUpdatedTime'],
      dtype='object')

In [ ]:
df.columns.tolist()


['ID',
 'SystemCodeNumber',
 'Capacity',
 'Latitude',
 'Longitude',
 'Occupancy',
 'VehicleType',
 'TrafficConditionNearby',
 'QueueLength',
 'IsSpecialDay',
 'LastUpdatedDate',
 'LastUpdatedTime']

In [ ]:
import pandas as pd

# Convert date
df['LastUpdatedDate'] = pd.to_datetime(df['LastUpdatedDate'], format='%d-%m-%Y')

# Extract day-of-year
df['Day'] = df['LastUpdatedDate'].dt.dayofyear

# Convert time to hour (0–23)
df['Time'] = pd.to_datetime(df['LastUpdatedTime'], format='%H:%M:%S').dt.hour

# Check dataset properties
print("Null values:\n", df.isnull().sum())
print("\nUnique Parking Lots (ID):", df['ID'].nunique())
print("Unique Days:", df['Day'].nunique())
print("Unique Time Steps:", df['Time'].nunique())


Null values:
 ID                        0
SystemCodeNumber          0
Capacity                  0
Latitude                  0
Longitude                 0
Occupancy                 0
VehicleType               0
TrafficConditionNearby    0
QueueLength               0
IsSpecialDay              0
LastUpdatedDate           0
LastUpdatedTime           0
Day                       0
Time                      0
dtype: int64

Unique Parking Lots (ID): 18368
Unique Days: 73
Unique Time Steps: 10


In [ ]:
df['occ_rate'] = df['Occupancy'] / df['Capacity']


In [ ]:
print(df['SystemCodeNumber'].nunique())
print(df['SystemCodeNumber'].head(20))


14
0     BHMBCCMKT01
1     BHMBCCMKT01
2     BHMBCCMKT01
3     BHMBCCMKT01
4     BHMBCCMKT01
5     BHMBCCMKT01
6     BHMBCCMKT01
7     BHMBCCMKT01
8     BHMBCCMKT01
9     BHMBCCMKT01
10    BHMBCCMKT01
11    BHMBCCMKT01
12    BHMBCCMKT01
13    BHMBCCMKT01
14    BHMBCCMKT01
15    BHMBCCMKT01
16    BHMBCCMKT01
17    BHMBCCMKT01
18    BHMBCCMKT01
19    BHMBCCMKT01
Name: SystemCodeNumber, dtype: object


In [ ]:
print(df['SystemCodeNumber'].nunique())
df['SystemCodeNumber'].value_counts().head(20)


14


,count
SystemCodeNumber,
BHMBCCMKT01,1312
BHMBCCTHL01,1312
BHMEURBRD01,1312
BHMMBMMBX01,1312
BHMNCPHST01,1312
BHMNCPNST01,1312
Broad Street,1312
Others-CCCPS105a,1312
Others-CCCPS119a,1312


In [ ]:
import pandas as pd

# Rename correct lot ID
df = df.rename(columns={'SystemCodeNumber': 'LotID'})

# Convert date
df['LastUpdatedDate'] = pd.to_datetime(df['LastUpdatedDate'], format='%d-%m-%Y')
df['Day'] = df['LastUpdatedDate'].dt.dayofyear

# Convert time (KEEP minutes)
df['Time'] = pd.to_datetime(df['LastUpdatedTime'], format='%H:%M:%S').dt.strftime('%H:%M')

# Feature: occupancy rate
df['occ_rate'] = df['Occupancy'] / df['Capacity']

# Check dataset structure
print("Null values:\n", df.isnull().sum())
print("\nUnique Parking Lots (LotID):", df['LotID'].nunique())
print("Unique Days:", df['Day'].nunique())
print("Unique Time Steps:", df['Time'].nunique())


Null values:
 ID                        0
LotID                     0
Capacity                  0
Latitude                  0
Longitude                 0
Occupancy                 0
VehicleType               0
TrafficConditionNearby    0
QueueLength               0
IsSpecialDay              0
LastUpdatedDate           0
LastUpdatedTime           0
Day                       0
Time                      0
occ_rate                  0
dtype: int64

Unique Parking Lots (LotID): 14
Unique Days: 73
Unique Time Steps: 275


In [ ]:
# Convert raw time
df['LastUpdatedTime'] = pd.to_datetime(df['LastUpdatedTime'], format='%H:%M:%S')

# Round to nearest 30-minute interval
df['Time'] = df['LastUpdatedTime'].dt.round('30min').dt.strftime('%H:%M')

print("Unique Time Steps:", df['Time'].nunique())
print(sorted(df['Time'].unique()))


Unique Time Steps: 19
['07:30', '08:00', '08:30', '09:00', '09:30', '10:00', '10:30', '11:00', '11:30', '12:00', '12:30', '13:00', '13:30', '14:00', '14:30', '15:00', '15:30', '16:00', '16:30']


In [ ]:
df['LastUpdatedTime'] = pd.to_datetime(df['LastUpdatedTime'], format='%H:%M:%S')


In [ ]:
df['minutes'] = df['LastUpdatedTime'].dt.hour * 60 + df['LastUpdatedTime'].dt.minute


In [ ]:
df['bin_minutes'] = (df['minutes'] // 30) * 30


In [ ]:
df['Time'] = pd.to_datetime(df['bin_minutes'], unit='m').dt.strftime('%H:%M')


In [ ]:
valid_times = [
    "08:00","08:30","09:00","09:30","10:00","10:30",
    "11:00","11:30","12:00","12:30","13:00","13:30",
    "14:00","14:30","15:00","15:30","16:00","16:30"
]

df = df[df['Time'].isin(valid_times)]


In [ ]:
print("Unique time steps:", df['Time'].nunique())
print(sorted(df['Time'].unique()))


Unique time steps: 18
['08:00', '08:30', '09:00', '09:30', '10:00', '10:30', '11:00', '11:30', '12:00', '12:30', '13:00', '13:30', '14:00', '14:30', '15:00', '15:30', '16:00', '16:30']


In [ ]:
df['occ_rate'] = df['Occupancy'] / df['Capacity']


#STEP 3 : MODEL 1


In [ ]:
ALPHA = 2


In [ ]:
def model1(prev_price, occupancy, capacity, alpha=2):
    occ_rate = occupancy / capacity
    new_price = prev_price + alpha * occ_rate
    return max(5, min(new_price, 40))   # avoid extreme prices


#STEP 4 : MODEL 2

In [ ]:
def compute_demand(row):
    # Assign weights
    w_occ = 1.2
    w_queue = 0.8
    w_traffic = -0.5
    w_event = 2.0

    # Vehicle type weights
    vehicle_weights = {
        'Car': 1.0,
        'Bike': 0.5,
        'Truck': 1.5
    }

    vweight = vehicle_weights.get(row['VehicleType'], 1.0)

    demand = 0
    demand += w_occ * (row['Occupancy'] / row['Capacity'])
    demand += w_queue * row['QueueLength']
    demand += w_traffic * row['TrafficConditionNearby']
    demand += w_event * row['IsSpecialDay']
    demand += vweight

    return demand


In [ ]:
df['TrafficConditionNearby'].unique()


array(['low', 'high', 'average'], dtype=object)

In [ ]:
# ---- Clean & compute demand (run this cell) ----
import pandas as pd
import numpy as np

# --- 1) Map traffic strings to numeric ---
traffic_map = {'low': 1, 'average': 2, 'high': 3}
df['TrafficConditionNearby'] = df['TrafficConditionNearby'].astype(str).str.lower().map(traffic_map)

# --- 2) Ensure numeric columns are numeric, coerce errors to NaN then fill with sensible defaults ---
numeric_cols = ['QueueLength', 'Occupancy', 'Capacity', 'IsSpecialDay', 'TrafficConditionNearby']
for c in numeric_cols:
    df[c] = pd.to_numeric(df[c], errors='coerce')

# Fill NaNs: for counts use 0, for Capacity use median (avoid dividing by zero)
df['QueueLength'] = df['QueueLength'].fillna(0)
df['Occupancy'] = df['Occupancy'].fillna(0)
if df['Capacity'].isna().any():
    median_capacity = int(df['Capacity'].median(skipna=True))
    df['Capacity'] = df['Capacity'].fillna(median_capacity)
df['IsSpecialDay'] = df['IsSpecialDay'].fillna(0)
df['TrafficConditionNearby'] = df['TrafficConditionNearby'].fillna(df['TrafficConditionNearby'].median())

# --- 3) Vehicle type mapping (case-insensitive) ---
vehicle_weights = {
    'car': 1.0,
    'bike': 0.5,
    'truck': 1.5
}
# Normalize strings then map; unknowns -> 1.0
df['VehicleType_clean'] = df['VehicleType'].astype(str).str.lower().str.strip()
df['VehicleTypeWeight'] = df['VehicleType_clean'].map(vehicle_weights).fillna(1.0)

# --- 4) Compute occupancy rate safely (avoid div-by-zero) ---
# If capacity is zero, replace with median capacity to avoid division errors
df['Capacity'] = df['Capacity'].replace(0, df['Capacity'].median())
df['occ_rate'] = df['Occupancy'] / df['Capacity']

# --- 5) Demand function (tunable weights) ---
w_occ = 1.2
w_queue = 0.8
w_traffic = -0.5     # negative if traffic reduces willingness to go there
w_event = 2.0

def compute_demand(row):
    # Base demand from occupancy rate
    demand = 0.0
    demand += w_occ * (row['occ_rate'])
    demand += w_queue * row['QueueLength']
    demand += w_traffic * row['TrafficConditionNearby']
    demand += w_event * row['IsSpecialDay']
    # add vehicle type contribution
    demand += row['VehicleTypeWeight']
    return demand

# Apply
df['demand'] = df.apply(compute_demand, axis=1)

# --- 6) Normalize demand to [0,1] safely ---
dmin = df['demand'].min()
dmax = df['demand'].max()
if pd.isna(dmin) or pd.isna(dmax) or dmax == dmin:
    # fallback: if constant or NaN, set normalized demand to 0.5
    df['norm_demand'] = 0.5
else:
    df['norm_demand'] = (df['demand'] - dmin) / (dmax - dmin)

# --- 7) Quick sanity prints ---
print("Traffic unique (mapped):", sorted(df['TrafficConditionNearby'].dropna().unique()))
print("Vehicle types (sample):", df['VehicleType_clean'].value_counts().head().to_dict())
print("Demand: min, mean, max ->", df['demand'].min(), df['demand'].mean(), df['demand'].max())
print("Norm_demand: min, mean, max ->", df['norm_demand'].min(), df['norm_demand'].mean(), df['norm_demand'].max())
print("\nSample rows (LotID, Time, occ_rate, QueueLength, Traffic, IsSpecialDay, VehicleTypeWeight, demand, norm_demand):")
display(df[['LotID','Time','occ_rate','QueueLength','TrafficConditionNearby','IsSpecialDay','VehicleTypeWeight','demand','norm_demand']].head(8))


Traffic unique (mapped): [np.int64(1), np.int64(2), np.int64(3)]
Vehicle types (sample): {'car': 10895, 'bike': 3555, 'truck': 1768, 'cycle': 1740}
Demand: min, mean, max -> 0.02287694974003468 4.682406237813641 14.566804017512233
Norm_demand: min, mean, max -> 0.0 0.3203762825790449 1.0

Sample rows (LotID, Time, occ_rate, QueueLength, Traffic, IsSpecialDay, VehicleTypeWeight, demand, norm_demand):


,LotID,Time,occ_rate,QueueLength,TrafficConditionNearby,IsSpecialDay,VehicleTypeWeight,demand,norm_demand
1,BHMBCCMKT01,08:00,0.110919,1,1,0,1.0,1.433102,0.096963
2,BHMBCCMKT01,08:30,0.138648,2,1,0,1.0,2.266378,0.154257
3,BHMBCCMKT01,09:30,0.185442,2,1,0,1.0,2.322530,0.158118
4,BHMBCCMKT01,09:30,0.259965,2,1,0,0.5,1.911958,0.129888
5,BHMBCCMKT01,10:00,0.306759,3,1,0,1.0,3.268111,0.223133
6,BHMBCCMKT01,10:30,0.379549,6,3,0,1.5,5.255459,0.359778
7,BHMBCCMKT01,11:00,0.428076,5,2,0,1.0,4.513692,0.308776
8,BHMBCCMKT01,11:30,0.448873,5,2,0,1.0,4.538648,0.310492


In [ ]:
df['demand'] = df.apply(compute_demand, axis=1)
df['norm_demand'] = (df['demand'] - df['demand'].min()) / (df['demand'].max() - df['demand'].min())


In [ ]:
def model2(norm_demand, base_price=10, lam=0.5):
    price = base_price * (1 + lam * norm_demand)
    return max(5, min(price, 20))     # bounded smooth price


#STEP 5 MODEL 3


In [ ]:
import numpy as np

def haversine(lat1, lon1, lat2, lon2):
    R = 6371
    dlat = np.radians(lat2 - lat1)
    dlon = np.radians(lon2 - lon1)
    a = (np.sin(dlat/2)**2
         + np.cos(np.radians(lat1))*np.cos(np.radians(lat2))*np.sin(dlon/2)**2)
    return 2 * R * np.arcsin(np.sqrt(a))


In [ ]:
def get_competitors(df, lot_id, radius_km=0.5):
    lat, lon = df[df['LocationID']==lot_id][['Lat','Lon']].iloc[0]
    neighbors = []
    for other_id in df['LocationID'].unique():
        if other_id == lot_id:
            continue
        lat2, lon2 = df[df['LocationID']==other_id][['Lat','Lon']].iloc[0]
        if haversine(lat, lon, lat2, lon2) <= radius_km:
            neighbors.append(other_id)
    return neighbors


In [ ]:
def model3(my_price, competitor_prices):
    if not competitor_prices:
        return my_price

    avg_comp = np.mean(competitor_prices)

    if my_price > avg_comp:      # overpriced
        return my_price - 2
    elif my_price < avg_comp:    # underpriced
        return my_price + 2
    else:
        return my_price



#STEP : Integrate with pathway



In [ ]:
!pip install pathway


In [ ]:
import pandas as pd
import numpy as np

# Load raw data
df = pd.read_csv("project/data/dataset.csv")

# Rename columns
df = df.rename(columns={
    'SystemCodeNumber': 'LotID'
})

# Date → Day
df['LastUpdatedDate'] = pd.to_datetime(df['LastUpdatedDate'], format='%d-%m-%Y')
df['Day'] = df['LastUpdatedDate'].dt.dayofyear

# Time → 30-min bins
df['LastUpdatedTime'] = pd.to_datetime(df['LastUpdatedTime'], format='%H:%M:%S')
minutes = df['LastUpdatedTime'].dt.hour * 60 + df['LastUpdatedTime'].dt.minute
df['Time'] = ((minutes // 30) * 30).apply(lambda x: f"{x//60:02d}:{x%60:02d}")

# Traffic mapping
traffic_map = {'low': 1, 'average': 2, 'high': 3}
df['Traffic'] = df['TrafficConditionNearby'].str.lower().map(traffic_map)

# Vehicle weights
vehicle_map = {'car': 1.0, 'bike': 0.5, 'truck': 1.5}
df['VehicleTypeWeight'] = df['VehicleType'].str.lower().map(vehicle_map)

# Occupancy rate
df['occ_rate'] = df['Occupancy'] / df['Capacity']

# Demand
df['demand'] = (
    1.2 * df['occ_rate']
    + 0.8 * df['QueueLength']
    - 0.5 * df['Traffic']
    + 2.0 * df['IsSpecialDay']
    + df['VehicleTypeWeight']
)

# Normalize demand
df['norm_demand'] = (df['demand'] - df['demand'].min()) / (df['demand'].max() - df['demand'].min())

# KEEP ONLY REQUIRED COLUMNS
df_clean = df[
    [
        'LotID',
        'Day',
        'Time',
        'Occupancy',
        'Capacity',
        'QueueLength',
        'Traffic',
        'IsSpecialDay',
        'VehicleTypeWeight',
        'norm_demand'
    ]
]

# Save cleaned dataset (overwrite if exists)
df_clean.to_csv("project/data/dataset_cleaned.csv", index=False)

print("Saved cleaned dataset")
print(df_clean.head())


Saved cleaned dataset
         LotID  Day   Time  Occupancy  Capacity  QueueLength  Traffic  \
0  BHMBCCMKT01  278  07:30         61       577            1        1   
1  BHMBCCMKT01  278  08:00         64       577            1        1   
2  BHMBCCMKT01  278  08:30         80       577            2        1   
3  BHMBCCMKT01  278  09:30        107       577            2        1   
4  BHMBCCMKT01  278  09:30        150       577            2        1   

   IsSpecialDay  VehicleTypeWeight  norm_demand  
0             0                1.0     0.096534  
1             0                1.0     0.096963  
2             0                1.0     0.154257  
3             0                1.0     0.158118  
4             0                0.5     0.129888  


In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv("project/data/dataset.csv")


In [ ]:
df = df.rename(columns={'SystemCodeNumber': 'LotID'})

df['LastUpdatedDate'] = pd.to_datetime(df['LastUpdatedDate'], format='%d-%m-%Y')
df['Day'] = df['LastUpdatedDate'].dt.dayofyear

df['LastUpdatedTime'] = pd.to_datetime(df['LastUpdatedTime'], format='%H:%M:%S')
minutes = df['LastUpdatedTime'].dt.hour * 60 + df['LastUpdatedTime'].dt.minute
df['Time'] = ((minutes // 30) * 30).apply(lambda x: f"{x//60:02d}:{x%60:02d}")


In [ ]:
traffic_map = {'low': 1, 'average': 2, 'high': 3}
df['Traffic'] = df['TrafficConditionNearby'].astype(str).str.lower().map(traffic_map)
df['Traffic'] = df['Traffic'].fillna(2)   # default = average


In [ ]:
vehicle_map = {'car': 1.0, 'bike': 0.5, 'truck': 1.5}

df['VehicleTypeClean'] = df['VehicleType'].astype(str).str.lower().str.strip()

df['VehicleTypeWeight'] = df['VehicleTypeClean'].map(vehicle_map)

# VERY IMPORTANT: fill missing weights
df['VehicleTypeWeight'] = df['VehicleTypeWeight'].fillna(1.0)

# Force numeric
df['VehicleTypeWeight'] = df['VehicleTypeWeight'].astype(float)


In [ ]:
df['occ_rate'] = df['Occupancy'] / df['Capacity']

df['demand'] = (
    1.2 * df['occ_rate']
    + 0.8 * df['QueueLength']
    - 0.5 * df['Traffic']
    + 2.0 * df['IsSpecialDay']
    + df['VehicleTypeWeight']
)

df['norm_demand'] = (
    (df['demand'] - df['demand'].min()) /
    (df['demand'].max() - df['demand'].min())
)


In [ ]:
df_clean = df[
    [
        'LotID',
        'Day',
        'Time',
        'Occupancy',
        'Capacity',
        'QueueLength',
        'Traffic',
        'IsSpecialDay',
        'VehicleTypeWeight',
        'norm_demand'
    ]
]


In [ ]:
df_clean.to_csv("project/data/dataset_cleaned.csv", index=False)


In [ ]:
!head -n 5 project/data/dataset_cleaned.csv


LotID,Day,Time,Occupancy,Capacity,QueueLength,Traffic,IsSpecialDay,VehicleTypeWeight,norm_demand
BHMBCCMKT01,278,07:30,61,577,1,1,0,1.0,0.09653418424333687
BHMBCCMKT01,278,08:00,64,577,1,1,0,1.0,0.09696317210073227
BHMBCCMKT01,278,08:30,80,577,2,1,0,1.0,0.15425688372176224
BHMBCCMKT01,278,09:30,107,577,2,1,0,1.0,0.15811777443832084


In [ ]:
!head -n 5 project/data/dataset_cleaned.csv



LotID,Day,Time,Occupancy,Capacity,QueueLength,Traffic,IsSpecialDay,VehicleTypeWeight,norm_demand
BHMBCCMKT01,278,07:30,61,577,1,1,0,1.0,0.09653418424333687
BHMBCCMKT01,278,08:00,64,577,1,1,0,1.0,0.09696317210073227
BHMBCCMKT01,278,08:30,80,577,2,1,0,1.0,0.15425688372176224
BHMBCCMKT01,278,09:30,107,577,2,1,0,1.0,0.15811777443832084


In [ ]:
import pathway as pw

class ParkingSchema(pw.Schema):
    LotID: str
    Day: int
    Time: str
    Occupancy: int
    Capacity: int
    QueueLength: int
    Traffic: int
    IsSpecialDay: int
    VehicleTypeWeight: float
    norm_demand: float

input_table = pw.io.csv.read(
    "project/data/dataset_cleaned.csv",
    schema=ParkingSchema,
    mode="static"
)

output_table = input_table.select(
    LotID=input_table.LotID,
    Time=input_table.Time,
    Price=10 * (1 + 0.5 * input_table.norm_demand)
)

output_table.show()


Column
    [0] Row
        [0] Markdown(str)
        [1] TooltipIcon(value='Immediate table p...)
    [1] Tabulator(disabled=True, height=400, page_size=10, pagination='local', show_index=False, sizing_mode='stretch_width', value=              ...)

In [ ]:
pw.run()


Output()

#PRICE SMOOTHING

In [ ]:
RAW_BASE_PRICE = 10
LAMBDA = 0.5

@pw.udf
def raw_price(norm_demand: float) -> float:
    price = RAW_BASE_PRICE * (1 + LAMBDA * norm_demand)
    return max(5, min(price, 20))


In [ ]:
priced_table = input_table.select(
    LotID=input_table.LotID,
    Day=input_table.Day,
    Time=input_table.Time,
    RawPrice=raw_price(input_table.norm_demand)
)


In [ ]:
pw.io.csv.write(
    priced_table,
    "project/data/raw_prices.csv"
)


In [ ]:
pw.run()

Output()

In [ ]:
import pandas as pd

df_prices = pd.read_csv("project/data/raw_prices.csv")
df_prices.head()


,LotID,Day,Time,RawPrice,time,diff
0,Others-CCCPS135a,310,08:00,10.774751,1767616336686,1
1,Shopping,279,11:00,12.048406,1767616336686,1
2,BHMEURBRD01,283,10:30,11.835999,1767616336686,1
3,BHMNCPHST01,319,10:00,11.654341,1767616336686,1
4,Others-CCCPS8,302,11:00,11.399162,1767616336686,1


In [ ]:
df_prices = df_prices.sort_values(
    by=['LotID', 'Day', 'Time']
).reset_index(drop=True)


In [ ]:
ALPHA = 0.3

df_prices['SmoothedPrice'] = (
    df_prices
    .groupby('LotID')['RawPrice']
    .transform(lambda x: x.ewm(alpha=ALPHA, adjust=False).mean())
)


In [ ]:
MAX_DELTA = 1.5  # max allowed price change per step

def cap_change(series):
    capped = [series.iloc[0]]
    for i in range(1, len(series)):
        prev = capped[-1]
        curr = series.iloc[i]
        capped.append(
            prev + max(-MAX_DELTA, min(MAX_DELTA, curr - prev))
        )
    return capped

df_prices['FinalPrice'] = (
    df_prices
    .groupby('LotID')['SmoothedPrice']
    .transform(cap_change)
)


In [ ]:
df_prices[['LotID', 'Time', 'RawPrice', 'SmoothedPrice', 'FinalPrice']].head(10)


,LotID,Time,RawPrice,SmoothedPrice,FinalPrice
0,BHMBCCMKT01,07:30,10.482671,10.482671,10.482671
1,BHMBCCMKT01,08:00,10.484816,10.483314,10.483314
2,BHMBCCMKT01,08:30,10.771284,10.569705,10.569705
3,BHMBCCMKT01,09:30,10.790589,10.635970,10.635970
4,BHMBCCMKT01,09:30,10.649440,10.640011,10.640011
5,BHMBCCMKT01,10:00,11.115666,10.782708,10.782708
6,BHMBCCMKT01,10:30,11.798889,11.087562,11.087562
7,BHMBCCMKT01,11:00,11.543880,11.224457,11.224457
8,BHMBCCMKT01,11:30,11.552459,11.322858,11.322858
9,BHMBCCMKT01,12:00,12.038765,11.537630,11.537630


#Competition Based Pricing

In [ ]:
import pandas as pd

df_raw = pd.read_csv("project/data/dataset.csv")

# Get unique location info per lot
lot_locations = (
    df_raw[['SystemCodeNumber', 'Latitude', 'Longitude']]
    .drop_duplicates()
    .rename(columns={'SystemCodeNumber': 'LotID'})
)

lot_locations.head()


,LotID,Latitude,Longitude
0,BHMBCCMKT01,26.144536,91.736172
1312,BHMBCCTHL01,26.144495,91.736205
2624,BHMEURBRD01,26.149020,91.739503
3936,BHMMBMMBX01,20.000035,78.000003
5248,BHMNCPHST01,26.140014,91.731000


In [ ]:
df_prices = pd.read_csv("project/data/raw_prices.csv")

# If you already added smoothed / final prices, load that file instead
# df_prices = pd.read_csv("project/data/final_prices.csv")

df_prices = df_prices.merge(lot_locations, on='LotID', how='left')


In [ ]:
import numpy as np

def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # km
    dlat = np.radians(lat2 - lat1)
    dlon = np.radians(lon2 - lon1)
    a = (
        np.sin(dlat / 2)**2
        + np.cos(np.radians(lat1))
        * np.cos(np.radians(lat2))
        * np.sin(dlon / 2)**2
    )
    return 2 * R * np.arcsin(np.sqrt(a))


In [ ]:
RADIUS_KM = 0.5


In [ ]:
def competitive_price_adjustment(df, radius_km=0.5, delta=0.8):
    prices = []

    for i, row in df.iterrows():
        lat1, lon1 = row['Latitude'], row['Longitude']

        # Compute distances to all others
        distances = haversine(
            lat1, lon1,
            df['Latitude'].values,
            df['Longitude'].values
        )

        # Nearby competitors (exclude self)
        mask = (distances <= radius_km) & (distances > 0)

        competitors = df.loc[mask, 'FinalPrice']

        if len(competitors) == 0:
            prices.append(row['FinalPrice'])
            continue

        avg_comp_price = competitors.mean()

        if row['FinalPrice'] > avg_comp_price:
            prices.append(row['FinalPrice'] - delta)
        else:
            prices.append(row['FinalPrice'] + delta)

    return prices


In [ ]:
df_prices.columns


Index(['LotID', 'Day', 'Time', 'RawPrice', 'time', 'diff', 'Latitude',
       'Longitude'],
      dtype='object')

In [ ]:
ALPHA = 0.3

df_prices = df_prices.sort_values(
    by=['LotID', 'Day', 'Time']
).reset_index(drop=True)

df_prices['SmoothedPrice'] = (
    df_prices
    .groupby('LotID')['RawPrice']
    .transform(lambda x: x.ewm(alpha=ALPHA, adjust=False).mean())
)


In [ ]:
MAX_DELTA = 1.5  # max price change per step

def cap_change(series):
    capped = [series.iloc[0]]
    for i in range(1, len(series)):
        prev = capped[-1]
        curr = series.iloc[i]
        capped.append(
            prev + max(-MAX_DELTA, min(MAX_DELTA, curr - prev))
        )
    return capped

df_prices['FinalPrice'] = (
    df_prices
    .groupby('LotID')['SmoothedPrice']
    .transform(cap_change)
)


In [ ]:
df_prices[['LotID', 'Time', 'RawPrice', 'SmoothedPrice', 'FinalPrice']].head(10)


,LotID,Time,RawPrice,SmoothedPrice,FinalPrice
0,BHMBCCMKT01,07:30,10.482671,10.482671,10.482671
1,BHMBCCMKT01,08:00,10.484816,10.483314,10.483314
2,BHMBCCMKT01,08:30,10.771284,10.569705,10.569705
3,BHMBCCMKT01,09:30,10.790589,10.635970,10.635970
4,BHMBCCMKT01,09:30,10.649440,10.640011,10.640011
5,BHMBCCMKT01,10:00,11.115666,10.782708,10.782708
6,BHMBCCMKT01,10:30,11.798889,11.087562,11.087562
7,BHMBCCMKT01,11:00,11.543880,11.224457,11.224457
8,BHMBCCMKT01,11:30,11.552459,11.322858,11.322858
9,BHMBCCMKT01,12:00,12.038765,11.537630,11.537630


In [ ]:
df_prices['CompetitivePrice'] = competitive_price_adjustment(
    df_prices,
    radius_km=0.5,
    delta=0.8
)

df_prices['CompetitivePrice'] = df_prices['CompetitivePrice'].clip(5, 25)


In [ ]:
df_prices[['LotID', 'Time', 'RawPrice', 'SmoothedPrice', 'FinalPrice','CompetitivePrice']].head(10)


,LotID,Time,RawPrice,SmoothedPrice,FinalPrice,CompetitivePrice
0,BHMBCCMKT01,07:30,10.482671,10.482671,10.482671,11.282671
1,BHMBCCMKT01,08:00,10.484816,10.483314,10.483314,11.283314
2,BHMBCCMKT01,08:30,10.771284,10.569705,10.569705,11.369705
3,BHMBCCMKT01,09:30,10.790589,10.635970,10.635970,11.435970
4,BHMBCCMKT01,09:30,10.649440,10.640011,10.640011,11.440011
5,BHMBCCMKT01,10:00,11.115666,10.782708,10.782708,11.582708
6,BHMBCCMKT01,10:30,11.798889,11.087562,11.087562,11.887562
7,BHMBCCMKT01,11:00,11.543880,11.224457,11.224457,12.024457
8,BHMBCCMKT01,11:30,11.552459,11.322858,11.322858,12.122858
9,BHMBCCMKT01,12:00,12.038765,11.537630,11.537630,12.337630


In [ ]:
df_prices

,LotID,Day,Time,RawPrice,time,diff,Latitude,Longitude,SmoothedPrice,FinalPrice,CompetitivePrice
0,BHMBCCMKT01,278,07:30,10.482671,1767616336678,1,26.144536,91.736172,10.482671,10.482671,11.282671
1,BHMBCCMKT01,278,08:00,10.484816,1767616336686,1,26.144536,91.736172,10.483314,10.483314,11.283314
2,BHMBCCMKT01,278,08:30,10.771284,1767616336678,1,26.144536,91.736172,10.569705,10.569705,11.369705
3,BHMBCCMKT01,278,09:30,10.790589,1767616336678,1,26.144536,91.736172,10.635970,10.635970,11.435970
4,BHMBCCMKT01,278,09:30,10.649440,1767616336678,1,26.144536,91.736172,10.640011,10.640011,11.440011
...,...,...,...,...,...,...,...,...,...,...,...
18363,Shopping,354,14:30,12.140154,1767616336678,1,26.150504,91.733531,12.156310,12.156310,12.156310
18364,Shopping,354,15:00,11.308621,1767616336686,1,26.150504,91.733531,11.902003,11.902003,11.902003
18365,Shopping,354,15:00,11.296803,1767616336678,1,26.150504,91.733531,11.720443,11.720443,11.720443
18366,Shopping,354,16:00,10.997924,1767616336686,1,26.150504,91.733531,11.503688,11.503688,11.503688


#Bookeh Visualization

In [ ]:
!pip install bokeh


In [ ]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource, Select
from bokeh.layouts import column
import pandas as pd

output_notebook()


In [ ]:
df_prices['Time_str'] = df_prices['Time']   # keep original


In [ ]:
DAY_TO_PLOT = df_prices['Day'].min()

df_day = df_prices[df_prices['Day'] == DAY_TO_PLOT]


In [ ]:
initial_lot = df_day['LotID'].unique()[0]


In [ ]:
def get_lot_data(lot_id):
    d = df_day[df_day['LotID'] == lot_id].sort_values('Time_str')
    return ColumnDataSource(d)

source = get_lot_data(initial_lot)


In [ ]:
p = figure(
    title=f"Dynamic Parking Price – Lot {initial_lot}",
    x_range=sorted(df_day['Time_str'].unique()),
    x_axis_label="Time",
    y_axis_label="Price",
    width=900,
    height=400
)

p.line(
    x='Time_str',
    y='CompetitivePrice',
    source=source,
    line_width=3
)

p.circle(
    x='Time_str',
    y='CompetitivePrice',
    source=source,
    size=6
)


GlyphRenderer(id='08ee8b3a-cb36-4748-8dda-262a7133f377', ...)

In [ ]:
from bokeh.models import CustomJS

lot_ids = sorted(df_day['LotID'].unique())

select = Select(
    title="Select Parking Lot:",
    value=initial_lot,
    options=lot_ids
)

callback = CustomJS(
    args=dict(source=source),
    code="""
    const data = source.data;
    const lot = cb_obj.value;

    fetch("")
    """
)


In [ ]:
LOT_TO_PLOT = initial_lot   # change this manually

df_plot = df_day[df_day['LotID'] == LOT_TO_PLOT].sort_values('Time_str')

p = figure(
    title=f"Dynamic Price for Parking Lot {LOT_TO_PLOT}",
    x_range=df_plot['Time_str'].tolist(),
    x_axis_label="Time",
    y_axis_label="Price",
    width=900,
    height=400
)

p.line(
    df_plot['Time_str'],
    df_plot['CompetitivePrice'],
    line_width=3
)

p.scatter(
    df_plot['Time_str'],
    df_plot['CompetitivePrice'],
    size=8
)

show(p)


ERROR:bokeh.core.validation.check:E-1019 (DUPLICATE_FACTORS): FactorRange must specify a unique list of categorical factors for an axis: duplicate factors found: '09:30', '13:00', '14:00'


In [ ]:
LOT_TO_PLOT = df_prices['LotID'].iloc[0]
DAY_TO_PLOT = df_prices['Day'].iloc[0]

df_plot = df_prices[
    (df_prices['LotID'] == LOT_TO_PLOT) &
    (df_prices['Day'] == DAY_TO_PLOT)
].copy()


In [ ]:
df_plot = df_plot.sort_values('Time_str')

# Ensure unique time values (important for Bokeh)
df_plot = df_plot.groupby('Time_str', as_index=False)['CompetitivePrice'].mean()


In [ ]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook

output_notebook()

p = figure(
    title=f"Dynamic Price for Parking Lot {LOT_TO_PLOT} (Day {DAY_TO_PLOT})",
    x_range=df_plot['Time_str'].tolist(),
    x_axis_label="Time",
    y_axis_label="Price",
    width=900,
    height=400
)

p.line(
    x=df_plot['Time_str'],
    y=df_plot['CompetitivePrice'],
    line_width=3
)

p.scatter(
    x=df_plot['Time_str'],
    y=df_plot['CompetitivePrice'],
    size=8
)

p.xaxis.major_label_orientation = 0.8

show(p)


#Report

# Dynamic Pricing for Urban Parking Lots

Capstone Project – Summer Analytics 2025  
Name: <Your Name> Shreyas  
Institute: <Your Institute> IIT Indore


Urban parking spaces are limited resources with highly variable demand.
Static pricing leads to either overcrowding or underutilization.

This project builds a real-time dynamic pricing engine for urban parking lots using demand signals, temporal patterns, and competitive behavior.

## Problem Statement


The objective is to design a pricing system that dynamically updates parking prices based on real-time demand, traffic conditions, special events, vehicle type, and nearby competitors.

The system must ensure:

Smooth and realistic price changes

Explainable logic

Bounded pricing behavior

Real-time simulation using Pathway

## Dataset Description


The dataset contains observations from 14 urban parking locations collected over 73 days, with multiple time snapshots per day.

Key Features:

Parking lot capacity and occupancy

Queue length

Vehicle type (car, bike, truck)

Nearby traffic condition

Special day indicator

Timestamp information

Raw timestamps were irregular and required preprocessing to align them into fixed 30-minute slots.

## Data Preprocessing


Data preprocessing involved:

Converting timestamps into fixed 30-minute intervals

Mapping categorical variables (traffic, vehicle type) into numeric representations

Handling missing or inconsistent values

Computing occupancy rate and normalized demand

## Model 1: Baseline Linear Pricing


Model 1 serves as a baseline where the price increases linearly with occupancy rate:

Pricet+1=Pricet+α⋅Capacity/Occupancy
	​
  
This model establishes a reference point but does not account for demand variability or competition.

## Model 2: Demand-Based Pricing


A composite demand function was constructed using multiple real-time features:

Occupancy rate

Queue length

Traffic condition

Special day indicator

Vehicle type weight

The demand score is normalized and mapped to price as:

𝑃
𝑟
𝑖
𝑐
𝑒
=
𝐵
𝑎
𝑠
𝑒
𝑃
𝑟
𝑖
𝑐
𝑒
⋅
(
1
+
𝜆
⋅
𝑁
𝑜
𝑟
𝑚
𝑎
𝑙
𝑖
𝑧
𝑒
𝑑
𝐷
𝑒
𝑚
𝑎
𝑛
𝑑
)
Price=BasePrice⋅(1+λ⋅NormalizedDemand)

This ensures prices respond to demand while remaining bounded.

## Price Smoothing


Raw dynamic prices can fluctuate sharply due to short-term demand spikes.
To ensure realistic pricing behavior, exponential smoothing was applied:

𝑃
𝑡
𝑠
𝑚
𝑜
𝑜
𝑡
ℎ
=
𝛼
𝑃
𝑡
+
(
1
−
𝛼
)
𝑃
𝑡
−
1
𝑠
𝑚
𝑜
𝑜
𝑡
ℎ
P
t
smooth
	​

=αP
t
	​

+(1−α)P
t−1
smooth
	​


Additionally, a maximum per-step price change constraint was enforced to prevent abrupt jumps.

## Model 3: Competition-Based Pricing


Real-world parking markets are competitive.

Using latitude–longitude data, nearby parking lots within a fixed radius were identified as competitors.

Pricing adjustments were made as follows:

If nearby competitors were cheaper → price slightly reduced

If nearby competitors were more expensive → price slightly increased

All adjustments were bounded to maintain stability.

## Real-Time Simulation with Pathway


Pathway was used to simulate real-time data ingestion and pricing updates.

Although static mode was used during development for debugging, the pipeline is compatible with streaming mode for real-time deployment.

## Visualization


The price evolution across time slots demonstrates:

Smooth transitions

Peak-hour price increases

Competitive adjustments

These trends validate the effectiveness of the pricing logic.

## Assumptions


Traffic levels mapped ordinally

Fixed competitor radius

Uniform base price

No behavioral feedback loop from users

## Conclusion


This project demonstrates a complete end-to-end dynamic pricing system for urban parking.

By combining demand modeling, smoothing, competition awareness, and real-time simulation, the system produces realistic and explainable prices suitable for real-world deployment.